# Unified Graph Construction API

This notebook demonstrates the unified TSGraph API for recurrence networks, ordinal partition networks, and visibility graphs as described in Varley & Sporns (2020) style construction.

All three graph types use the same TSGraph container and `draw_tsgraph` renderer.

## Installation

```bash
pip install ts2net
```


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ts2net.viz import (
    build_recurrence_graph,
    build_ordinal_partition_graph,
    build_visibility_graph,
    draw_tsgraph,
    optimal_lag,
    optimal_dim,
)

np.random.seed(42)


## Example 1: Recurrence Networks (ε-sweep)

Recurrence networks use phase space embedding to capture temporal dynamics. The epsilon parameter controls the recurrence threshold.


In [ ]:
# Create a simple time series
t = np.linspace(0, 4*np.pi, 100)
x = np.sin(t) + 0.1 * np.random.randn(100)

# Build recurrence graphs with different epsilon values
epsilons = [0.1, 0.2, 0.3]
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, eps in enumerate(epsilons):
    tsgraph = build_recurrence_graph(
        x,
        embed_dim=3,
        delay=1,
        eps=eps,
        eps_mode='fraction_max',
        return_pos=True,
    )
    
    draw_tsgraph(
        tsgraph,
        ax=axes[idx],
        show=False,
        node_size=8,
        edge_alpha=0.1,
        color_by='time',
    )
    axes[idx].set_title(f'ε = {eps} ({tsgraph.graph.number_of_edges()} edges)', fontsize=11)

plt.suptitle('Recurrence Networks: Epsilon Sweep', fontsize=13)
plt.tight_layout()
plt.show()

print(f"Created {len(epsilons)} recurrence graphs with different epsilon values")


## Example 2: Ordinal Partition Network

Ordinal partition networks capture symbolic dynamics by encoding the relative ordering of values in embedded vectors.


In [ ]:
# Create a time series with clear patterns
t = np.linspace(0, 4*np.pi, 150)
x = np.sin(t) + 0.05 * np.random.randn(150)

# Use optimal parameters
tau = optimal_lag(x)
d = optimal_dim(x, delay=tau, dim_range=(3, 6))

print(f"Optimal delay (τ): {tau}")
print(f"Optimal dimension (d): {d}")

tsgraph = build_ordinal_partition_graph(
    x,
    embed_dim=d,
    delay=tau,
    directed=True,
    weighted=True,
    return_pos=True,
)

fig, ax = draw_tsgraph(
    tsgraph,
    show=False,
    node_size=15,
    edge_alpha=0.3,
    color_by='degree',
    cmap='plasma',
)
ax.set_title(f'Ordinal Partition Network (d={d}, τ={tau})\n{tsgraph.graph.number_of_nodes()} patterns, {tsgraph.graph.number_of_edges()} transitions', 
             fontsize=11)
plt.show()

print(f"Created OPN with {tsgraph.graph.number_of_nodes()} patterns")


## Example 3: Visibility Graphs Comparison

Compare Horizontal Visibility Graph (HVG) and Natural Visibility Graph (NVG) using the unified API.


In [ ]:
x = np.sin(np.linspace(0, 4*np.pi, 100)) + 0.1 * np.random.randn(100)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# HVG
tsgraph_hvg = build_visibility_graph(x, kind='hvg', directed=False)
draw_tsgraph(tsgraph_hvg, ax=axes[0], show=False, node_size=15, color_by='time')
axes[0].set_title(f'HVG ({tsgraph_hvg.graph.number_of_edges()} edges)', fontsize=11)

# NVG
tsgraph_nvg = build_visibility_graph(x, kind='nvg', directed=False)
draw_tsgraph(tsgraph_nvg, ax=axes[1], show=False, node_size=15, color_by='time')
axes[1].set_title(f'NVG ({tsgraph_nvg.graph.number_of_edges()} edges)', fontsize=11)

plt.suptitle('Visibility Graphs: HVG vs NVG', fontsize=13)
plt.tight_layout()
plt.show()

print(f"HVG: {tsgraph_hvg.graph.number_of_edges()} edges")
print(f"NVG: {tsgraph_nvg.graph.number_of_edges()} edges")
